# Market Value Item

### What is market value?

All of the previously discussed items have focused on different collections of securities (positions, accounts, and portfolios). Now we're going to take another step forward and discuss market value. Market value is generally the price that an asset of any type (e.g., equity, fixed income, derivatives, fx) would fetch in the marketplace. It is also the total value of the securities in which you hold a position. Depending on the security type, market value can be simple (i.e., for equities, the value of a unit is well-known) or more complex with various options (i.e., for derivatives, there are a few ways to determine and/or state what the market value of your position is).

For our purpose we'll stick with simple market value calculations where market value is equal to some unit multiplier times your positions

For example, you could have a position and current share price/unit market value like below:

My position is ***2000*** shares of ***MSFT USD***. The current price of ***MSFT USD*** is ***19.00***. This gives my position a 
market value of **38,000 USD**!

### Problem Definition

We want to extend our classes to allow for querying of the market values for various securities at different levels of aggregation. We want this classes to have to the ability below functionality:

***Security***
- Query the current market value of the security

***Positions*** 
- Query the current market value for the current position

***Account***
- Query the market value for a given set of security names or security objects. These objects should act as filters to what should be included in calculating the accounts net market value. Any securities not present in the account but the filter can be ignored
- Query the market value for the entire account. This should return a net market value of all positions in the account.

***Portfolio***
- Query the market value for the entire portfolio. This requires that security market values be atomic in your calculation method per query.
- Query the market value for a given set of securities/account combinations. If securities are present they should act as a filter on the accounts to be queried. If accounts are present they should act as a filter on the accounts under query. Handle scenarios where securities or accounts given aren't present without causing exceptions.
- **Create tests for the above portfolio features**

### Provided Tools

#### *Data Source*

To simulate prices, you can utilize the given class priceData which can be imported from generators.priceDataGenerator. This class is a singleton and will generate prices each time a query is given. These prices are stored internally in the class for testing in a replay mode. For a given security if the security name contains "Equity" or "eqty" (case insensitive) the market value will always be positive. To query the price of a security you can use the object's get_current_price method. Below is an example of this

```python
#Data Souce
from generators.price_data_generator import PriceData

price_Data = PriceData()

cur_price = price_data.get_current_price("IBM US Equity")
```

#### *Solution Interface*

The below interface show the new functionality you should add to your existing classes.

```python
#filename security_interface.py
#Security Class Interface
class SecurityInterface():
    #...
    #Previous implementation above
    def get_current_market_value(self) -> float:
```

```python
#filename position_interface.py
#Position Class Interface
class PositionInterface():
    #...
    #Previous implementation above
    def get_current_market_value(self) -> float:
        pass
```

```python
#filename account_interface.py
#Account Class Interface
class AccountInterface():
    #...
    #Previous implementation above
    
    def get_current_market_value(self) -> float:
        pass

    def get_current_filtered_market_value(self, securities: Set) -> float:
        pass
```

```python
#filename portfolioInterface.py
#Portfolio Class Interface
class portfolio_interface():
    #...
    #Previous implementation above
    
    def get_current_market_value(self) -> dict:
        pass

    def get_current_filtered_market_value(self, securities: Set, accountNames: Set[str]) -> float:
        pass
```

#### *Testing*

Once you have completed & saved your solution you can run the test file to validate that your solution works as expected. For the test to run the following need to be true.
- Saved code to files to their respective .py's in the implementation folders. **Remember to copy the current implementation as saving will overwrite your previous implementation!** 

In [ ]:
#%%writefile ../implementations/securitySolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements securityInterface & allows for the management of a security

In [ ]:
#%%writefile ../implementations/positionSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements positionInterface & allows for the management of a position

In [ ]:
#%%writefile ../implementations/accountSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements accountInterface & allows for the management of an account

In [ ]:
#%%writefile ../implementations/portfolioSolution.py 
#Uncomment line above & run cell to save solution
#TODO Define class that implements portFolioInterface & allows for the management of a portfolio

In [ ]:
#%conda install ipytest
#%pip install ipytest
import ipytest

ipytest.autoconfig()

In [ ]:
%%ipytest -qq

from generators.price_data_generator import PriceData
import implementations.security_solution
import implementations.position_solution
import implementations.account_solution
import implementations.portfolio_solution

import importlib
importlib.reload(implementations.portfolio_solution)
importlib.reload(implementations.security_solution)
importlib.reload(implementations.position_solution)
importlib.reload(implementations.account_solution)


def create_portfolio_accounts():
    account_a_positions = [
        implementations.position_solution.Position("IBM US Equity", 530),
        implementations.position_solution.Position("TSLA US Equity", 1120),
        implementations.position_solution.Position("NVDA US Equity", 7421),
    ]

    account_b_positions = [
        implementations.position_solution.Position("IBM US Equity", 201),
        implementations.position_solution.Position("MSFT US Equity", 400),
        implementations.position_solution.Position("NVDA US Equity", 300),
        implementations.position_solution.Position("DLTA US Equity", 623),
    ]

    account_a = implementations.account_solution.Account(
        account_a_positions, "Account A"
    )
    account_b = implementations.account_solution.Account(
        account_b_positions, "Account B"
    )

    accounts = {"Account A": account_a, "Account B": account_b}

    return accounts


def test_security_value_gather():
    # GIVEN
    security_name = "TSLA US Equity"
    data_source = PriceData()
    data_source.clear_price_history()

    # WHEN
    test_obj = implementations.security_solution.Security(security_name)
    ## TODO: @Reviewers, we are calcualting MV and assigning it to price. This looks wrong.
    current_price = test_obj.get_current_market_value()

    # EXPECT
    assert (
        current_price
        == data_source.get_security_price_data_list(security_name)[-1]
    )


def test_position_market_value():
    # GIVEN
    expected_name = "IBM US Equity"
    expected_position_amount = 1000
    data_source = PriceData()
    data_source.clear_price_history()

    # WHEN
    test_obj = implementations.position_solution.Position(
        expected_name, expected_position_amount
    )
    MV = test_obj.get_current_market_value()
    lastest_expected_mv = (
        expected_position_amount
        * data_source.get_security_price_data_list(expected_name)[-1]
    )

    # EXPECT
    assert lastest_expected_mv == MV


def test_security_search_account_mv():
    # GIVEN
    data_source = PriceData()
    data_source.clear_price_history()
    expected_account_positions = [
        implementations.position_solution.Position("IBM US Equity", 530),
        implementations.position_solution.Position("TSLA US Equity", 1120),
        implementations.position_solution.Position("NVDA US Equity", 7421),
    ]
    search_securities_list = [
        "IBM US Equity",
        implementations.security_solution.Security("NVDA US Equity"),
        "MSFT US Equity",
    ]
    search_securities_tuple = [["IBM US Equity", 530], ["NVDA US Equity", 7421]]
    test_obj = implementations.account_solution.Account(
        expected_account_positions, "Test Account"
    )

    # WHEN
    market_Value = test_obj.get_current_filtered_market_value(
        search_securities_list
    )
    expected_mv = 0
    for sec_tuple in search_securities_tuple:
        if sec_tuple[1] != 0:
            expected_mv += (
                sec_tuple[1]
                * data_source.get_security_price_data_list(sec_tuple[0])[-1]
            )
    # EXPECT
    assert expected_mv == market_Value


def test_total_account_mv():
    # GIVEN
    expected_account_positions = [
        implementations.position_solution.Position("IBM US Equity", 530),
        implementations.position_solution.Position("TSLA US Equity", 1120),
        implementations.position_solution.Position("NVDA US Equity", 7421),
    ]
    data_source = PriceData()
    data_source.clear_price_history()
    test_obj = implementations.account_solution.Account(
        expected_account_positions, "Test Account"
    )

    # WHEN
    market_value = test_obj.get_current_market_value()
    expected_mv = 0
    for pos in expected_account_positions:
        expected_mv += (
            pos.get_position()
            * data_source.get_security_price_data_list(
                pos.get_security().get_name()
            )[-1]
        )
    # EXPECT
    assert expected_mv == market_value


def test_total_portfolio_mv():
    portfolio_name = "TestPortfolio"
    accounts = create_portfolio_accounts()
    position_map_total = {}

    for acc in accounts.values():
        for pos in acc.get_all_positions():
            if pos.get_security().get_name() in position_map_total:
                position_map_total[pos.get_security().get_name()].add_position(
                    pos.get_position()
                )
            else:
                position_map_total[
                    pos.get_security().get_name()
                ] = implementations.position_solution.Position(
                    pos.get_security(), pos.get_position()
                )

    data_source = PriceData()
    data_source.clear_price_history()
    test_obj = implementations.portfolio_solution.Portfolio(
        portfolio_name, accounts.values()
    )

    mv_total = test_obj.get_current_market_value()
    expected_mv_total = 0
    for pos in position_map_total.values():
        expected_mv_total += (
            pos.get_position()
            * data_source.get_security_price_data_list(
                pos.get_security().get_name()
            )[-1]
        )
    assert mv_total == expected_mv_total


def test_filtered_portfolio_mvs():
    # GIVEN
    portfolio_name = "TestPortfolio"
    accounts = create_portfolio_accounts()

    position_map_total_acc = {}
    position_map_total_security = {}
    position_map_total_security_acc = {}
    security_filter = "IBM US Equity"
    acc_filter = "Account B"
    for acc in accounts.values():
        for pos in acc.get_all_positions():
            if pos.get_security().get_name() == security_filter:
                if pos.get_security().get_name() in position_map_total_security:
                    position_map_total_security[
                        pos.get_security().get_name()
                    ].add_position(pos.get_position())
                else:
                    position_map_total_security[
                        pos.get_security().get_name()
                    ] = implementations.position_solution.Position(
                        pos.get_security().get_name(), pos.get_position()
                    )

            if acc.get_name() == acc_filter:
                if pos.get_security().get_name() in position_map_total_acc:
                    position_map_total_acc[
                        pos.get_security().get_name()
                    ].add_position(pos.get_position())
                else:
                    position_map_total_acc[
                        pos.get_security().get_name()
                    ] = implementations.position_solution.Position(
                        pos.get_security().get_name(), pos.get_position()
                    )

                if pos.get_security().get_name() == security_filter:
                    if (
                        pos.get_security().get_name()
                        in position_map_total_security_acc
                    ):
                        position_map_total_security_acc[
                            pos.getSecurityName()
                        ].add_position(pos.get_position())
                    else:
                        position_map_total_security_acc[
                            pos.get_security().get_name()
                        ] = implementations.position_solution.Position(
                            pos.get_security().get_name(), pos.get_position()
                        )

    data_source = PriceData()
    data_source.clear_price_history()
    test_obj = implementations.portfolio_solution.Portfolio(
        portfolio_name, accounts.values()
    )

    # WHEN
    mv_acc = test_obj.get_current_filtered_market_value([acc_filter], [])
    expected_mv_acc = 0
    for pos in position_map_total_acc.values():
        expected_mv_acc += (
            pos.get_position()
            * data_source.get_security_price_data_list(
                pos.get_security().get_name()
            )[-1]
        )
    data_source = PriceData()
    data_source.clear_price_history()

    mv_security = test_obj.get_current_filtered_market_value(
        [], [security_filter]
    )
    expected_mv_security = 0
    for pos in position_map_total_security.values():
        expected_mv_security += (
            pos.get_position()
            * data_source.get_security_price_data_list(
                pos.get_security().get_name()
            )[-1]
        )
    data_source = PriceData()
    data_source.clear_price_history()

    mv_acc_security = test_obj.get_current_filtered_market_value(
        [acc_filter], [security_filter]
    )
    expected_mv_acc_security = 0
    for pos in position_map_total_security_acc.values():
        expected_mv_acc_security += (
            pos.get_position()
            * data_source.get_security_price_data_list(
                pos.get_security().get_name()
            )[-1]
        )
    data_source = PriceData()
    data_source.clear_price_history()

    # EXPECT
    assert mv_acc == expected_mv_acc
    assert mv_acc_security == expected_mv_acc_security
    assert mv_security == expected_mv_security